In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load and preprocess the data
data = pd.read_csv('x6perfect_reseampled.csv')
X = data.iloc[:, :-1].values / 255.0
y = data.iloc[:, -1].values

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVM model
svm = SVC(kernel='rbf', random_state=42)

# Define hyperparameters for grid search
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'class_weight': [None, 'balanced']
}

# Perform grid search
grid_search = GridSearchCV(
    svm,
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1,
    scoring=['accuracy', 'precision_macro', 'recall_macro'],
    refit='accuracy'
)

# Fit the grid search
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Train the best model
best_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test_scaled)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print(f"Test accuracy: {accuracy:.4f}")
print(f"Test precision: {precision:.4f}")
print(f"Test recall: {recall:.4f}")

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Function to predict emotion for new RGB values
def predict_emotion(rgb_values):
    rgb_array = np.array(rgb_values).reshape(1, -1) / 255.0
    rgb_scaled = scaler.transform(rgb_array)
    prediction_encoded = best_model.predict(rgb_scaled)
    prediction = label_encoder.inverse_transform(prediction_encoded)
    return prediction[0]

# Example usage
new_rgb = [229, 0, 13, 225, 225, 255, 253, 166, 74]
predicted_emotion = predict_emotion(new_rgb)
print(f"\nPredicted Emotion: {predicted_emotion}")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Test accuracy: 0.8757
Test precision: 0.8763
Test recall: 0.8814

Classification Report:
              precision    recall  f1-score   support

           0       0.78      1.00      0.88        14
           1       0.91      1.00      0.95        21
           2       1.00      1.00      1.00        16
           3       0.85      1.00      0.92        11
           4       0.93      0.87      0.90        15
           5       0.75      0.92      0.83        13
           6       0.87      0.95      0.91        21
           7       0.80      1.00      0.89        16
           8       0.88      1.00      0.94        15
           9       1.00      1.00      1.00        15
          10       0.95      1.00      0.97        18
          11       0.78      1.00      0.88        18
          12       1.00      1.00      1.00         9
          13       1.00      1.00      1.00        11
          14       0.90      0.82     